In [1]:
import numpy as np
import sys

sys.path.append('..')
from utils import load_diabetes, train_val_test_split

data_utils = load_diabetes()

data_utils.shape

data_list=[]

f = open("diabete.txt",encoding = "utf-8")
a_list=f.readlines()
f.close()
for line in a_list:
    line1=line.replace('\n', '')
    line2=list(line1.split(' '))
    y=float(line2[0])
    x= [float(line2[i].split(':')[1]) for i in (1,2,3,4,5,6,7,8)]
    data_list.append(x+[y])


data_array_1=np.array(data_list)[:,:-1]
data_array_0=np.ones((data_array_1.shape[0],1))
data_array_2=data_array_1*data_array_1
data_array_3=np.empty((data_array_1.shape[0],0))

for i in range(data_array_1.shape[1]):
    for j in range(data_array_1.shape[1]):
        if i<j:
            data_array_i=data_array_1[:,i]*data_array_1[:,j]
            data_array_i=np.reshape(data_array_i,(-1,1))
            data_array_3=np.hstack((data_array_3,data_array_i))

data_array_4=np.reshape(np.array(data_list)[:,-1],(-1,1))
data=np.hstack((data_array_0,data_array_1,data_array_2,data_array_3,data_array_4))

n_train = 500
n_val = 150

metrics = []
variables = []

hparams = {
    'gam': 5,
    # 'eta': 0.1,
    'alpha': 0.01,
    'beta': 0.05,
    'epsilon': 1e-5,
    'max_iters_outer': 80,
    'max_iters_inner': 100,
    'M': 1e-3,
    't': 1e-4
}

# epochs = 80
plot_results = True

In [2]:
c0, w0, b0, xi0 = 10 * np.ones(n_train), np.random.randn(45), np.random.randn(), np.random.randn(n_train)
from algorithms import barrier_blo
for seed in range(10):

    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(data, seed, n_train, n_val)
    datasets = {
        'x_train': x_train, 'y_train': y_train, 'x_val': x_val, 
        'y_val': y_val, 'x_test': x_test, 'y_test': y_test
    }
    
    problem = barrier_blo.SVM_Problem(datasets)
    algo = barrier_blo.Barrier_BLO(problem, hparams)
    
    # metrics_seed, variables_seed = barrier_blo(x_train, y_train, x_val, y_val, x_test, y_test, hparams, epochs)
    metrics_seed, c, w, b, xi, grad_norm = algo.upper_loop(c0, w0, b0, xi0, hparams)
    metrics.append(metrics_seed)
    # variables.append([c, w, b, xi])

train_acc = np.array([[x['train_acc'] for x in metrics] for metrics in metrics])
val_acc = np.array([[x['val_acc'] for x in metrics] for metrics in metrics])
test_acc = np.array([[x['test_acc'] for x in metrics] for metrics in metrics])
val_loss = np.array([[x['val_loss'] for x in metrics] for metrics in metrics])
test_loss = np.array([[x['test_loss'] for x in metrics] for metrics in metrics])

time_computation = np.array([[x['time_computation'] for x in metrics] for metrics in metrics])


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jiaxiangli/anaconda3/envs/blo/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jiaxiangli/anaconda3/envs/blo/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/jiaxiangli/anaconda3/envs/blo/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jiaxiangli/anaconda3/envs/blo/lib/python3.10/site-packages/traitlets/config/application.py",

    Inner loop PGD total iter: 100, grad norm: 5927.214323728376
hessian_y shape: (546, 546), jacobian shape: (500, 546), np.block([grad_w, grad_b, grad_xi]) shape: (546,)
Upper iter: 0, grad norm of hyperfunction: 256.50240757106474
Epoch [0/80]: val acc: 0.01 val loss: 0.00 test acc: 0.01 test loss: 0.00


UnboundLocalError: local variable 'w' referenced before assignment